In [1]:
#!pip install ultralytics opencv-python pillow

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split
from PIL import Image
import yaml
import numpy as np
import cv2
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
import timm
from itertools import product
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit

In [4]:
path = os.getcwd()
print(path)

#os.chdir(path)
#file_log = open(path + "/mensagem_final_classificar_V2.txt", "a")

/content


In [5]:
# === Configurações Gerais ===
NUM_CLASSES = 15
INPUT_SIZE = 224  # conforme modelo Small treinado em 224×224
BATCH_SIZE = 32 # Padrão: 32
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATASET_LEAF = "/content/drive/MyDrive/TCC/Datasets/Imagens Folhas/Especies" # Dataset de folhas usado
DATASET_BARK = "/content/drive/MyDrive/TCC/Datasets/Imagens tronco/EspeciesCascas" # Dataset de cascas usado

In [6]:
#  ===Dataset customizado ===
class ImageFolderDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        # Assumindo estrutura: root_dir/class_x/imagename.jpg
        self.samples = []
        classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name:i for i,cls_name in enumerate(classes)}
        for cls_name in classes:
            cls_path = os.path.join(root_dir, cls_name)
            if not os.path.isdir(cls_path):
                continue
            count = 0
            for fname in os.listdir(cls_path):
                if fname.lower().endswith(('.png','.jpg','.jpeg','bmp')):
                    self.samples.append((os.path.join(cls_path, fname), self.class_to_idx[cls_name]))
                    count += 1
            print(f"{cls_name}: {count} imagens.")
        # embaralhar?
        np.random.seed(42)
        np.random.shuffle(self.samples)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        # leitura via OpenCV
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # redimensionar
        img = cv2.resize(img, (INPUT_SIZE, INPUT_SIZE))
        # converter para float32 e normalizar [0,1]
        img = img.astype(np.float32) / 255.0
        # talvez normalização adicional conforme modelo (media/std)
        # usando valores padrão ImageNet
        mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
        std  = np.array([0.229, 0.224, 0.225], dtype=np.float32)
        img = (img - mean) / std
        # mudar de H×W×C para C×H×W
        img = np.transpose(img, (2,0,1))
        img_tensor = torch.from_numpy(img)
        label_tensor = torch.tensor(label, dtype=torch.long)
        return img_tensor, label_tensor

In [7]:
def stratified_split(dataset, test_split=0.1, valid_split=0.2, seed=42):
    """
    Divide o dataset em treino, validação e teste mantendo a proporção das classes.

    Args:
        dataset: Dataset com atributo 'samples' [(path, label), ...]
        test_split: Proporção do teste em relação ao total (padrão: 0.1 = 10%)
        valid_split: Proporção da validação em relação ao (treino+validação) (padrão: 0.2 = 20%)
        seed: Seed para reprodutibilidade

    Returns:
        train_ds, valid_ds, test_ds: Subsets estratificados

    Nota: Com os valores padrão, a divisão real é:
        - Treino: 72% do total
        - Validação: 18% do total
        - Teste: 10% do total
    """
    # Extrair os rótulos
    labels = [label for _, label in dataset.samples]

    # Primeiro, separar Teste
    sss1 = StratifiedShuffleSplit(n_splits=1, test_size=test_split, random_state=seed)
    train_valid_idx, test_idx = next(sss1.split(np.zeros(len(labels)), labels))

    # Agora, separar Validação dentro do conjunto de treino+validação
    labels_train_valid = np.array(labels)[train_valid_idx]
    sss2 = StratifiedShuffleSplit(n_splits=1, test_size=valid_split, random_state=seed)
    train_idx, valid_idx = next(sss2.split(np.zeros(len(labels_train_valid)), labels_train_valid))

    # Reindexar para o dataset original
    train_idx = np.array(train_valid_idx)[train_idx]
    valid_idx = np.array(train_valid_idx)[valid_idx]

    # Criar Subsets
    train_ds = torch.utils.data.Subset(dataset, train_idx)
    valid_ds = torch.utils.data.Subset(dataset, valid_idx)
    test_ds  = torch.utils.data.Subset(dataset, test_idx)

    return train_ds, valid_ds, test_ds

In [8]:
# === Função principal para verificar os Datasets ===
def run_datasets(dataset_root, valid_split=0.2, test_split=0.1):
    # carregar dataset completo
    #global full_ds, train_ds, valid_ds, test_ds
    full_ds = ImageFolderDataset(dataset_root)
    total_len = len(full_ds)
    test_len  = int(total_len * test_split)
    valid_len = int((total_len - test_len) * valid_split)
    train_len = total_len - test_len - valid_len
    #train_ds, valid_ds, test_ds = random_split(full_ds, [train_len, valid_len, test_len], generator=torch.Generator().manual_seed(42))
    train_ds, valid_ds, test_ds = stratified_split(full_ds, test_split=test_split, valid_split=valid_split, seed=42)

    print(f"\nDataset total: {total_len} imagens")
    print(f"Classes detectadas ({len(full_ds.class_to_idx.keys())}): {full_ds.class_to_idx.keys()}")
    print(f"Treino: {train_len} | Validação: {valid_len} | Teste: {test_len}")

In [11]:
# === Função Main ===
if __name__ == "__main__":
    try:
        print("\n--------------- Análise dos Datasets ---------------\nInício...")
        print("\n--------------- Dataset de Folhas ---------------")
        run_datasets(DATASET_LEAF, valid_split=0.2, test_split=0.1)
        print("\n--------------- Dataset de Cascas ---------------")
        run_datasets(DATASET_BARK, valid_split=0.2, test_split=0.1)
        print("\n...Fim\n")
    except KeyboardInterrupt:
        print("Programa encerrado via terminal...")


--------------- Análise dos Datasets ---------------
Início...

--------------- Dataset de Folhas ---------------
Abacateiro: 42 imagens.
Araca: 83 imagens.
Brinco de Indio: 79 imagens.
Cajueiro: 76 imagens.
Carvalho: 30 imagens.
Caterete: 30 imagens.
Cerejeira: 30 imagens.
Coite: 30 imagens.
Fruta do conde: 54 imagens.
Grevilha: 47 imagens.
Jambolao: 30 imagens.
Laranja Champanhe: 31 imagens.
Louro Pardo: 31 imagens.
Pau Brasil: 78 imagens.
Peroba Rosa: 30 imagens.

Dataset total: 701 imagens
Classes detectadas (15): dict_keys(['Abacateiro', 'Araca', 'Brinco de Indio', 'Cajueiro', 'Carvalho', 'Caterete', 'Cerejeira', 'Coite', 'Fruta do conde', 'Grevilha', 'Jambolao', 'Laranja Champanhe', 'Louro Pardo', 'Pau Brasil', 'Peroba Rosa'])
Treino: 505 | Validação: 126 | Teste: 70

--------------- Dataset de Cascas ---------------
Abacateiro: 34 imagens.
Araca: 32 imagens.
Brinco de Indio: 32 imagens.
Cajueiro: 32 imagens.
Carvalho: 32 imagens.
Caterete: 34 imagens.
Cerejeira: 33 imagens.
Coi